In [26]:
import re
import yaml

def spice_to_yaml(spice_file):
    components = []

    with open(spice_file, 'r') as file:
        for line in file:
            line = line.strip()
            if line and not line.startswith(('*', '.param', '+')):
                if line.startswith('.MODEL'):
                    parts = re.split(r'\s+', line)
                    model_name = parts[1].strip('"')
                    model_params = {}
                    for param in parts[2:]:
                        param_parts = param.split('=')
                        if len(param_parts) == 2:
                            param_name, param_value = param_parts
                            model_params[param_name.strip('"')] = param_value.strip('{}')
                    components.append({
                        'type': 'model',
                        'name': model_name,
                        'parameters': model_params
                    })
                elif line.startswith('X'):
                    parts = re.split(r'\s+', line)
                    component_type = parts[0]
                    component_name = parts[0]
                    if len(parts) > 2:
                        component_nodes = parts[1:1+len(parts)-3]
                    else:
                        component_nodes = []
                    component_params = {}
                    for param in parts[1+len(parts)-3:]:
                        param_parts = param.split('=')
                        if len(param_parts) == 2:
                            param_name, param_value = param_parts
                            component_params[param_name.strip('"')] = param_value.strip('{}')
                    components.append({
                        'type': component_type,
                        'name': component_name,
                        'nodes': component_nodes,
                        'parameters': component_params
                    })
                else:
                    parts = re.split(r'\s+', line)
                    if len(parts) > 1:
                        component_type = parts[0]
                        component_name = parts[0] + parts[1]
                        component_nodes = parts[1:3]  # Keep node names as strings
                        try:
                            component_value = float(parts[3])
                        except (ValueError, IndexError) as e:
                            print(f"Error parsing line: {line}")
                            continue

                        components.append({
                            'type': component_type,
                            'name': component_name,
                            'nodes': component_nodes,
                            'value': component_value
                        })

    return {'components': components}

def main():
    spice_file = 'Root Element.spi'  # Change this to your SPICE file name
    yaml_data = spice_to_yaml(spice_file)
    
    # Writing YAML data to a file
    with open('output.yaml', 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False)

if __name__ == "__main__":
    main()
